In [1]:
from ollama import chat, ChatResponse
import textwrap
from IPython.display import Markdown, display
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine

In [7]:
def anomyser(text):
    analyzer = AnalyzerEngine()

    results = analyzer.analyze(text=text,
                           entities=["PHONE_NUMBER", "CREDIT_CARD", "PERSON", "EMAIL_ADDRESS", "IN_PAN", "IN_AADHAAR"],
                           language='en')
    print(results)

    anonymizer = AnonymizerEngine()

    anonymized_text = anonymizer.anonymize(text=text,analyzer_results=results)

    print(anonymized_text)
    return anonymized_text

In [3]:
content = """
Subject: Request to Update My Account Details

Hello Team,

Please update my account information as discussed.

Here are my details:

- Full Name: Rajesh Kumar
- Mobile Number: 98765-43210
- Email Address: rajesh.kumar.test@samplemail.com
- PAN Number: ABCDE1234F
- Aadhaar Number: 1234-5678-9012
- Credit Card Number: 4111-1111-1111-1111
- Expiry Date: 09/29
- CVV: 123
- Billing Address: 45, MG Road, Bangalore, Karnataka – 560001

Please confirm once the update is complete.

Thanks & Regards,
Rajesh Kumar

analyse this email is spam or not
"""

In [12]:
pii_content = anomyser(content)

[type: EMAIL_ADDRESS, start: 207, end: 239, score: 1.0, type: CREDIT_CARD, start: 320, end: 339, score: 1.0, type: PERSON, start: 148, end: 169, score: 0.85, type: PERSON, start: 498, end: 510, score: 0.85, type: PHONE_NUMBER, start: 178, end: 189, score: 0.75]
text: 
Subject: Request to Update My Account Details

Hello Team,

Please update my account information as discussed.

Here are my details:

- Full Name: <PERSON> Number: <PHONE_NUMBER>
- Email Address: <EMAIL_ADDRESS>
- PAN Number: ABCDE1234F
- Aadhaar Number: 1234-5678-9012
- Credit Card Number: <CREDIT_CARD>
- Expiry Date: 09/29
- CVV: 123
- Billing Address: 45, MG Road, Bangalore, Karnataka – 560001

Please confirm once the update is complete.

Thanks & Regards,
<PERSON>

analyse this email is spam or not

items:
[
    {'start': 465, 'end': 473, 'entity_type': 'PERSON', 'text': '<PERSON>', 'operator': 'replace'},
    {'start': 293, 'end': 306, 'entity_type': 'CREDIT_CARD', 'text': '<CREDIT_CARD>', 'operator': 'replace'},
   

In [14]:
print(pii_content.text)


Subject: Request to Update My Account Details

Hello Team,

Please update my account information as discussed.

Here are my details:

- Full Name: <PERSON> Number: <PHONE_NUMBER>
- Email Address: <EMAIL_ADDRESS>
- PAN Number: ABCDE1234F
- Aadhaar Number: 1234-5678-9012
- Credit Card Number: <CREDIT_CARD>
- Expiry Date: 09/29
- CVV: 123
- Billing Address: 45, MG Road, Bangalore, Karnataka – 560001

Please confirm once the update is complete.

Thanks & Regards,
<PERSON>

analyse this email is spam or not



In [15]:
messages = [
    {
        'role': 'user',
        'content':pii_content.text ,
    },
]

In [16]:
print(messages)

[{'role': 'user', 'content': '\nSubject: Request to Update My Account Details\n\nHello Team,\n\nPlease update my account information as discussed.\n\nHere are my details:\n\n- Full Name: <PERSON> Number: <PHONE_NUMBER>\n- Email Address: <EMAIL_ADDRESS>\n- PAN Number: ABCDE1234F\n- Aadhaar Number: 1234-5678-9012\n- Credit Card Number: <CREDIT_CARD>\n- Expiry Date: 09/29\n- CVV: 123\n- Billing Address: 45, MG Road, Bangalore, Karnataka – 560001\n\nPlease confirm once the update is complete.\n\nThanks & Regards,\n<PERSON>\n\nanalyse this email is spam or not\n'}]


In [17]:
response: ChatResponse = chat(model='deepseek-r1:1.5b', messages=messages)

text_output = response.message.content

display(Markdown(text_output))

The email appears to be a legitimate request for an account update rather than spam. The inclusion of standard account details suggests genuine interest in updating profile information, which is common in such contexts. While there are specific requests (CVV number, expiry date), these don't indicate spam practices. Therefore, it's likely not spam but a typical request for account updates.